In [216]:
import pandas as pd

## Indice de Precios de Consumo. Base 2021

Este conjunto de datos recoge los índices del IPC por comunidades autónomas, incluyendo el índice general y los grupos ECOICOP, lo que permite analizar la evolución del nivel de precios y su distribución territorial. Antes de su uso en el análisis exploratorio, será necesario realizar tareas de limpieza y selección de variables para garantizar la comparabilidad y relevancia de la información.  

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=50913)


In [217]:
df = pd.read_csv("./data/indice_precios_consumo.csv", sep= ";")
df = df[df["Comunidades y Ciudades Autónomas"] != "Nacional"] #Dejamos solo las comunidades autonomas
df = df[df["Tipo de dato"] == "Índice"]
df["año"] = df["Periodo"].str[:4].astype(int)
df.Total =df.Total.str.replace(",", "." ).astype(float)

In [218]:
df_anual = (df.groupby(["Comunidades y Ciudades Autónomas", "Grupos ECOICOP", "año"])["Total"].mean().reset_index())

In [219]:
wide = df_anual.pivot_table(
    index=["Comunidades y Ciudades Autónomas", "año"],
    columns="Grupos ECOICOP",
    values="Total",      # <-- CAMBIAR por el nombre real de la columna numérica
    aggfunc="first"      # o "mean" si hubiese duplicados
).reset_index()

In [220]:
df= wide.copy()
df["Comunidades y Ciudades Autónomas"] = df["Comunidades y Ciudades Autónomas"].str[3:].replace(r",.*$", "", regex=True).str.strip()
df.to_csv("./data/ipc_anual.csv", index=False)

## PIB per cápita  
 
El PIB per cápita es un indicador económico que relaciona el valor total de la producción de una región con su población, y se utiliza como aproximación al nivel medio de riqueza y capacidad económica de los territorios. En este análisis se empleará como variable base para comparar comunidades autónomas y estudiar su evolución temporal, sirviendo como punto de partida para el análisis del resto de indicadores socioeconómicos.

[Bibliografia](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736167628&menu=resultados&idp=1254735576581#_tabs-1254736158133)

In [221]:
df = pd.read_excel("./data/pr_cre_pr.xlsx",sheet_name="Tabla_2")
df_pib_pc = df.melt(id_vars="Comunidad Autónoma",var_name="año",value_name="pib_pc")
df_pib_pc["Comunidad Autónoma"] = df_pib_pc["Comunidad Autónoma"].str.title().str.replace(r",.*$", "", regex=True).str.strip()

## Renta disponible bruta de los hogares (per capita)

La renta disponible bruta de los hogares refleja el ingreso efectivo del que disponen los hogares para el consumo y el ahorro, una vez descontados impuestos y cotizaciones sociales y sumadas las transferencias recibidas. Este indicador resulta especialmente relevante para complementar el PIB per cápita, ya que ofrece una aproximación más directa al poder adquisitivo real de la población en cada comunidad autónoma.  

[Bibliografia](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736167628&menu=resultados&idp=1254735576581#_tabs-1254736195584)

In [222]:
df = pd.read_excel("./data/rentahogd24.xlsx",sheet_name="Tabla_2")
df_renta_pc = df.melt(id_vars="Comunidad Autónoma",var_name="año",value_name="renta_pc")
df_renta_pc["Comunidad Autónoma"] = df_renta_pc["Comunidad Autónoma"].str.title().str.replace(r",.*$", "", regex=True).str.strip()

### Integración de datos (PIB per capita y Renta disponible bruta per capita)

En esta etapa se realiza la unificación de los distintos conjuntos de datos mediante un proceso de merge, utilizando como claves comunes la comunidad autónoma y el año. Esta integración permite construir un DataFrame único que combina indicadores económicos y sociales, facilitando su análisis conjunto y la comparación territorial y temporal.

In [223]:
merge = df_pib_pc.merge(df_renta_pc,on=["Comunidad Autónoma","año"],how="left")

## Población

Se incorpora la población por comunidad autónoma y año con el objetivo de contextualizar los indicadores económicos y permitir análisis adicionales, como la validación de valores per cápita y el estudio de la evolución demográfica en relación con la renta y el PIB.  

[Bibliografia](https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=02003.px&L=0)


In [224]:
df = pd.read_csv("./data/poblacion.csv", sep= ";", dtype=str)
df = df.drop(columns= "Total Nacional")
df = df[~df["Comunidades y Ciudades Autónomas"].isna()]
df = df[df["Edad (año a año)"] == "TOTAL EDADES"]
df = df[df["Españoles/Extranjeros"] == "Españoles"]
df = df[df["Sexo"] == "Ambos sexos"]
df = df.drop(columns=["Edad (año a año)","Españoles/Extranjeros","Sexo"])
df = df.rename(columns={"Comunidades y Ciudades Autónomas":"Comunidad Autónoma","Periodo":"año","Total":"poblacion"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df = df[df["año"]>"1999"]
df["año"] = df["año"].astype(int) 
df_poblacion = df.copy()
df_poblacion.año.dtypes

dtype('int64')

### Integración de datos (Merge y Población)

In [225]:
merge1 = merge.merge(df_poblacion,on=["Comunidad Autónoma","año"],how="left")

## Población con gasto elevado en vivienda

Este indicador mide el porcentaje de población que destina una proporción elevada de sus ingresos al gasto en vivienda, y se utiliza como aproximación a la **presión económica asociada al acceso y mantenimiento de la vivienda**. Su incorporación permite complementar los indicadores de renta y precios, aportando una dimensión directa del esfuerzo económico de los hogares por comunidad autónoma.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69497&L=0)

In [226]:
df = pd.read_csv("./data/gasto_elevado_vivienda.csv", sep=";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"gasto_elevado_vivienda(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_elevado_vivienda = df.copy()

### Integración de datos (Merge1 y Población con gasto elevado en vivienda)

In [227]:
merge2 = merge1.merge(df_elevado_vivienda,on=["Comunidad Autónoma","año"],how="left")

## Población con falta de espacio en la vivienda

Este indicador (en %) estima la proporción de población que vive en viviendas con **hacinamiento o espacio insuficiente**. Se incorpora para añadir una dimensión material de la calidad de vida vinculada a las condiciones habitacionales, complementando renta, IPC y presión de gasto en vivienda.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69488&L=0)

In [228]:
df = pd.read_csv("./data/falta_espacio_vivienda.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"falta_espacio_vivienda(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_falta_espacio_vivienda = df.copy()

### Integración de datos (Merge2 y Población con falta de espacio en la vivienda)

In [229]:
merge3 = merge2.merge(df_falta_espacio_vivienda,on=["Comunidad Autónoma","año"],how="left")

### Retrasos en los pagos

Este indicador mide el porcentaje de población que ha experimentado **retrasos en el pago de gastos básicos** (como vivienda, suministros o préstamos), y se utiliza como proxy de **vulnerabilidad y seguridad económica**. Su incorporación permite complementar los indicadores de renta y condiciones materiales, aportando una medida directa de dificultades financieras de los hogares por comunidad autónoma.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69536&L=0)

In [230]:
df = pd.read_csv("./data/Retrasos_pagos.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año" , "Total":"retrasos_pagos(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_retraso_pago = df.copy()

### Integración de datos (Merge3 y Población con falta de espacio en la vivienda)

In [231]:
merge4 = merge3.merge(df_retraso_pago,on=["Comunidad Autónoma","año"],how="left")

## Renta media y mediana por unidad de consumo

Este indicador recoge la **renta media y la renta mediana por unidad de consumo**, permitiendo aproximar el nivel de ingresos de los hogares y su distribución. La comparación entre ambos valores resulta útil para identificar **asimetrías y posibles desigualdades internas** dentro de cada comunidad autónoma, complementando la renta disponible bruta y el PIB per cápita.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=68338&L=0)

In [232]:
df = pd.read_csv("./data/Renta_media_mediana.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()

In [233]:
df_renta_wide = (df
    .pivot_table(index=["Comunidad Autónoma", "año"],
                 columns="Renta media y mediana",
                 values="Total",
                 aggfunc="first")
    .reset_index()
)

In [234]:
df_renta_wide = df_renta_wide.rename(columns={"Renta media por unidad de consumo":"renta_media","Renta mediana por unidad de consumo":"renta_mediana"})

### Grafico de media y mediana por comunidad autonoma 

In [235]:
import matplotlib.pyplot as plt

def plot_renta_media_mediana(df, comunidad):
    df_c = (
        df[df["Comunidad Autónoma"] == comunidad]
        .sort_values("año")
    )
    
    plt.figure()
    plt.plot(df_c["año"], df_c["renta_media"], marker="o", label="Renta media")
    plt.plot(df_c["año"], df_c["renta_mediana"], marker="o", label="Renta mediana")
    
    plt.title(f"Renta media y mediana por unidad de consumo – {comunidad}")
    plt.xlabel("Año")
    plt.ylabel("Euros por unidad de consumo")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [236]:
# comunidades = df["Comunidad Autónoma"].unique()
# for comunidad in comunidades:
#     plot_renta_media_mediana(df_renta_wide,comunidad)

### Integración de datos (Merge4 y Renta media y mediana por unidad de consumo)

In [237]:
merge5 = merge4.merge(df_renta_wide,on=["Comunidad Autónoma","año"],how="left")

## Tasa de riesgo de pobreza (umbral fijo 2008)

Este indicador mide el porcentaje de población cuya renta se sitúa por debajo de un **umbral de pobreza fijo establecido en 2008**, lo que permite analizar la **evolución del poder adquisitivo real** de los hogares a lo largo del tiempo. Su uso facilita comparaciones temporales sin el efecto de cambios en el umbral relativo, complementando los indicadores de renta, precios y condiciones materiales.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69366&L=0)

In [238]:
df = pd.read_csv("./data/Tasa_riesgo_pobreza.csv", sep= ";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"riesgo_pobreza(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_riesgo_pobreza = df.copy()

### Integración de datos (Merge5 y Renta media y mediana por unidad de consumo)

In [239]:
merge6 = merge5.merge(df_riesgo_pobreza,on=["Comunidad Autónoma","año"],how="left")
merge6["año"] = merge6["año"].astype(int)

## Dificultades para llegar a fin de mes

Este indicador recoge el porcentaje de población que declara tener **algún grado de dificultad para llegar a fin de mes** (con mucha dificultad, con dificultad o con cierta dificultad). Se utiliza como medida sintética de **estrés económico percibido**, complementando los indicadores objetivo

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69461&L=0)

In [240]:
df = pd.read_csv("./data/Dificultades_llegar_fin_mes.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"dificultad_fin_mes(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df["año"] = df["año"].astype(int)
df["dificultad_fin_mes(%)"] = df["dificultad_fin_mes(%)"].str.replace(",",".").astype(float)

In [241]:
df_dificultad = (df.groupby(["año","Comunidad Autónoma"],as_index=False)["dificultad_fin_mes(%)"].sum())

### Integración de datos (Merge6 y Dificultades para llegar a fin de mes)

In [242]:
merge7 = merge6.merge(df_dificultad,on=["Comunidad Autónoma","año"],how="left")

## Desigualdad de ingresos (S80/S20)

El indicador S80/S20 mide la desigualdad de ingresos como el cociente entre la renta total del 20 % de la población con mayores ingresos y la del 20 % con menores ingresos. Valores más altos indican mayor desigualdad. Su incorporación permite contextualizar los indicadores de renta y pobreza, aportando una dimensión distributiva al análisis de las condiciones económicas.

PIB ↑ + S80/S20 ↑
→ crecimiento desigual

Renta media ↑ + S80/S20 estable
→ mejora más homogénea

Renta mediana baja + S80/S20 alto
→ desigualdad estructural

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69378&L=0)

In [243]:
df = pd.read_csv("./data/Desigualdad_S80S20.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"desigualdad_ing(S80/S20)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_desig_ingresos = df.copy()

### Integración de datos (Merge7 y Desigualdad de ingresos (S80/S20) 

In [244]:
merge8 = merge7.merge(df_desig_ingresos,on=["Comunidad Autónoma","año"],how="left")

### Incapacidad de hacer frente a gastos económicos imprevistos

Este indicador mide el porcentaje de población que declara no poder afrontar un **gasto económico imprevisto**, y se utiliza como aproximación a la **capacidad de resiliencia financiera** de los hogares. Su incorporación permite complementar los indicadores de renta, pobreza y dificultad económica, aportando una medida directa de vulnerabilidad ante shocks económicos.

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=69526&L=0)

In [245]:
df = pd.read_csv("./data/gastos_imprevistos.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"inc_gastos_imprevistos(%)"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df_imprevistos = df.copy()

### Integración de datos (Merge8 e Incapacidad de hacer frente a gastos económicos imprevistos)

In [246]:
merge9 = merge8.merge(df_imprevistos,on=["Comunidad Autónoma","año"],how="left")

### Seguridad física y personal

Para aproximar la dimensión de seguridad física y personal se incorpora la **tasa de criminalidad**, que refleja la incidencia de delitos en cada comunidad autónoma. Este indicador se considera más adecuado para el análisis de calidad de vida que la tasa de homicidios, dado que presenta mayor variabilidad territorial y temporal en el contexto español, permitiendo una mejor comparación entre comunidades y a lo largo del tiempo.  
La tasa de criminalidad es un indicador que mide la frecuencia de delitos en relación con la población, para poder comparar territorios con distinto tamaño poblacional (delitos cada 1000 habitantes).

[Bibliografia](https://www.ine.es/jaxiT3/Tabla.htm?t=74615&L=0)

In [247]:
df = pd.read_csv("./data/tasa_homicidio_criminalidad.csv", sep =";")
df = df[df["Comunidad autónoma"] != "Total Nacional"]
df = df.rename(columns={"Comunidad autónoma":"Comunidad Autónoma","Periodo":"año","Total":"tasa_criminalidad"})
df["Comunidad Autónoma"] = df["Comunidad Autónoma"].str[3:].replace(r",.*$", "", regex=True).str.strip().str.title()
df = df[df["Indicador de calidad de vida"] == "Tasa de criminalidad"]
df = df.drop(columns= "Indicador de calidad de vida")
df_criminalidad = df.copy()

# Integración de datos (Merge9 y Tasa de criminalidad)- Datafreme final por el momento

In [250]:
df = merge9.merge(df_criminalidad,on=["Comunidad Autónoma","año"],how="left")
df_periodo = df[(df["año"] > 2007) & (df["año"] < 2023)].copy()

In [251]:
df_periodo.to_csv("./data/pib_vs_calidad_vida.csv", index=False)